### Plot your own Mass-radius diagram

The mass-radius diagram (as seen in the lecture) is a way to visualize how the compositions varies among the planet population. 

**Start out by running all cells in the notebook (Shift+Enter or Ctrl+Enter) and have a look at the plot!**

_Credit for models: “Growth Model Interpretation of Planet Size Distribution”. Li Zeng, Stein B. Jacobsen, Dimitar D. Sasselov, Michail I. Petaev, Andrew Vanderburg, Mercedes Lopez-Morales, Juan Perez-Mercader, Thomas R. Mattsson, Gongjie Li, Matthew Z. Heising, Aldo S. Bonomo, Mario Damasso, Travis A. Berger, Hao Cao, Amit Levi, Robin Wordsworth. PNAS, 2019, 116 (20) 9723-9728. DOI: 10.1073/pnas.181295116 (https://www.pnas.org/content/116/20/9723)_

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

### Table of confirmed exoplanets with radius and RV mass

In [ ]:
# Load planets from the Exoplanet Archive with error bars better than 50%
all_planets = pd.read_csv('https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&'
            'select=pl_name,pl_masse,pl_masseerr1,pl_masseerr2,pl_rade,pl_radeerr1,pl_radeerr2&'
            'where=pl_tranflag>0')
all_planets = all_planets.dropna()
planets = all_planets
planets = planets.loc[(planets['pl_masseerr1']/planets['pl_masse']<0.5) | (-planets['pl_masseerr2']/planets['pl_masse']<0.5)]
planets = planets.loc[(planets['pl_radeerr1']/planets['pl_rade']<0.5) | (-planets['pl_radeerr2']/planets['pl_rade']<0.5)]

In [ ]:
def plot_planet(mass, radius, mass_err, rad_err, name=None, color='grey', alpha='1.0'):
    """A function that plots a single planet in the MR diagram"""
    plt.errorbar(mass, radius, xerr=mass_err, yerr=rad_err, 
                 fmt='o', color=color, alpha=1.0, markersize=5, markeredgecolor='none')
    if name:
        plt.text(mass, radius, ' '+name, color='k', clip_on=True, verticalalignment='top', fontsize=8)

def plot_all_planets(planets):
    """A function that plots (Mass,Radius) with errorbars for all planets in table"""
    plt.errorbar(planets['pl_masse'], planets['pl_rade'], 
                 xerr=(planets['pl_masseerr2'], -planets['pl_masseerr1']),
                 yerr=(planets['pl_radeerr2'], -planets['pl_radeerr1']),
                 fmt='o', color='grey', alpha=0.3, markersize=5, markeredgecolor='none')

def plot_named_planet(name, color='red'):
    """A function that plots one planet in the MR diagram"""
    planet = all_planets.loc[all_planets['pl_name'] == name]
    if len(planet) == 0:
        print('Not found: '+name)
        return
    plot_planet(planet['pl_masse'], planet['pl_rade'], 
             (planet['pl_masseerr2'], -planet['pl_masseerr1']),
             (planet['pl_radeerr2'], -planet['pl_radeerr1']), 
             name=planet['pl_name'].values[0], color=color)

### Solar system planets

In [ ]:
def plot_solar_system(labels=False):
    """Plot (Mass,Radius) for all solar system planets"""
    solar_system = [
        {'name': 'Mercury', 'mass': 0.0553, 'radius': 0.3829},
        {'name': 'Mars',    'mass': 0.107,  'radius': 0.532},
        {'name': 'Venus',   'mass': 0.815,  'radius': 0.9499},
        {'name': 'Earth',   'mass': 1,      'radius': 1},
        {'name': 'Neptune', 'mass': 17.147, 'radius': 3.865},
        {'name': 'Uranus',  'mass': 14.536, 'radius': 3.981},
        {'name': 'Saturn',  'mass': 95.162, 'radius': 9.14},
        {'name': 'Jupiter', 'mass': 317.83, 'radius': 10.97},
    ]
    for planet in solar_system:
        plt.plot(planet['mass'], planet['radius'], '^', color='green', zorder=10, markersize=5, markeredgecolor='none')
        if labels:
            plt.text(planet['mass'], planet['radius'], ' '+planet['name'], clip_on=True, verticalalignment='top', fontsize=8)

### Planet composition models

In [ ]:
def plot_model(path, label=None, col=None):
    modeldata = pd.read_csv(path, names=('mass', 'radius'), delimiter=r'\t', engine='python')
    plt.plot(modeldata['mass'], modeldata['radius'], label=label, color=col)

In [ ]:
def plot_models():
    plot_model('massradius/Earthlike2h300K1mbar.txt', '2% H$_2$ + H$_2$O/rocky', col='palevioletred')
    plot_model('massradius/Earthlike2h500K1mbar.txt', col='palevioletred')
    plot_model('massradius/Earthlike2h700K1mbar.txt', col='palevioletred')
    plot_model('massradius/massradius_100percentH2O_300K_1mbar.txt', '100% H$_2$O', col='blue')
    plot_model('massradius/massradius_100percentH2O_500K_1mbar.txt', col='blue')
    plot_model('massradius/massradius_100percentH2O_700K_1mbar.txt', col='blue')
    plot_model('massradius/massradius_50percentH2O_300K_1mbar.txt', '50% H$_2$O / 50% rocky', col='green')
    plot_model('massradius/massradius_50percentH2O_500K_1mbar.txt', col='green')
    plot_model('massradius/massradius_50percentH2O_700K_1mbar.txt', col='green')
    plot_model('massradius/massradiusmgsio3.txt', '100% MgSiO$_3$ (rock)', col='red')
    plot_model('massradius/massradiusEarthlikeRocky.txt', '33% Fe (Earth-like rocky)', col='orange')
    plot_model('massradius/massradiusFe.txt', '100% Fe', col='purple')

### Build the plot

In [ ]:
# Set plot size
plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['figure.dpi'] = 150

plot_all_planets(planets)
plot_solar_system()

# Decorate plot
plt.xscale('log')
plt.gca().xaxis.set_major_formatter(ScalarFormatter())
plt.xlabel('Mass ($M_\oplus$)');
plt.ylabel('Radius ($R_\oplus$)');
plt.ylim((0,23))

# Uncomment to show labels for solar system planets
#plot_solar_system(labels=True)

# Uncomment and fill in mass and radius for WASP-52b
#plot_planet(<mass>, <radius>, <mass_err>, <rad_err>, color='purple', name='WASP-52 b') 

# Uncomment to zoom in on rocky planets
#plt.xlim((0.5, 55)); plt.ylim((0.8,4.5));
#plot_models()
#plt.legend(loc='upper left');

# Uncomment to plot specific planets
#plot_named_planet('K2-18 b')
#plot_named_planet('K2-38 b')

# Uncomment to zoom in on TRAPPIST system
#plt.xlim((0.5, 10)); plt.ylim((0.8,2));
#plot_named_planet('TRAPPIST-1 b', color='orange')
#plot_named_planet('TRAPPIST-1 c', color='orange')
#plot_named_planet('TRAPPIST-1 d', color='orange')
#plot_named_planet('TRAPPIST-1 e', color='orange')
#plot_named_planet('TRAPPIST-1 f', color='orange')
#plot_named_planet('TRAPPIST-1 g', color='orange')

- The green triangles are the planets in our Solar system. Can you recognize which ones are which? Uncomment the line that shows the labels (remove the # in front) and re-run the cell.

- Note how most gas giants seem to be much more puffy than our own Jupiter and Saturn. How can that be?

- Now uncomment the line that plots in WASP-52 b, for which you just measured the mass and the radius. Re-run the cell. Is it where you would expect it to be?

- Let's zoom in on the lower left region. Uncomment the three lines and re-run the cell. You can now see the composition curves, as shown in the lecture. Multiple lines of same color shows composition at different temperatures. Uncomment the lines that plot named planets and re-run again.

- A year ago, the planet K2-18 b made it to the news as a "potentially habitable super-Earth". What can you say about the composition? How Earth-like is it?

- The TRAPPIST system is a system that will be studied intensively with the upcoming space telescope JWST. Which ones are likely to have more water-rich atmospheres?